# Object Detection Project

## Introduction
This project briefly explores multiple machine learning models tailor-made for realtime object detection.
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam venenatis enim in ex dictum sagittis. Vestibulum convallis ultricies lectus, vitae condimentum tortor laoreet vitae. Vivamus eget felis tempus, mattis ex quis, auctor libero. Suspendisse tincidunt augue in purus fringilla, id efficitur nunc porttitor. Donec auctor nibh a egestas dapibus. Donec erat elit, blandit in quam ac, rutrum porttitor metus. Morbi ut velit vitae nunc accumsan venenatis sed vitae enim. Proin sollicitudin sit amet ligula at ornare. Pellentesque ornare libero ut quam sollicitudin finibus. Sed mollis a urna in semper. Suspendisse sollicitudin condimentum sollicitudin.

Integer eget aliquet mi. Pellentesque a venenatis nunc. Quisque eget libero ultrices, vestibulum ex et, laoreet dui. In hendrerit ipsum vitae magna efficitur, in facilisis tellus efficitur. Duis leo sem, efficitur ac velit sed, egestas consequat augue. Mauris nec egestas mauris. Etiam vitae urna sollicitudin, ullamcorper felis nec, porttitor neque. Nulla vitae arcu at erat consequat vulputate nec vel metus. Quisque dui tortor, molestie vitae cursus vitae, pellentesque efficitur elit. Etiam eleifend tortor nibh, lacinia semper eros venenatis a. Duis sed tristique lorem. Donec interdum nulla id libero sodales aliquam. Donec laoreet, quam id rutrum aliquet, felis magna ullamcorper odio, eu consectetur sem nunc non eros. Aliquam fringilla arcu at augue suscipit, nec laoreet tortor faucibus. Nullam turpis tellus, pulvinar a rhoncus vitae, mattis vel ligula.

## Requirements
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nam venenatis enim in ex dictum sagittis. Vestibulum convallis ultricies lectus, vitae condimentum tortor laoreet vitae. Vivamus eget felis tempus, mattis ex quis, auctor libero. Suspendisse tincidunt augue in purus fringilla, id efficitur nunc porttitor. Donec auctor nibh a egestas dapibus. Donec erat elit, blandit in quam ac, rutrum porttitor metus. Morbi ut velit vitae nunc accumsan venenatis sed vitae enim. Proin sollicitudin sit amet ligula at ornare. Pellentesque ornare libero ut quam sollicitudin finibus. Sed mollis a urna in semper. Suspendisse sollicitudin condimentum sollicitudin.

In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import tensorflow as tf

In [ ]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

print("[INFO] Starting streaming...")
pipeline.start(config)
print("[INFO] Camera ready.")

In [ ]:
print("[INFO] Loading model...")
PATH_TO_CKPT = "frozen_inference_graph.pb"

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v1.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.compat.v1.import_graph_def(od_graph_def, name='')
    sess = tf.compat.v1.Session(graph=detection_graph)

In [ ]:
# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')
# code source of tensorflow model loading:
# https://www.geeksforgeeks.org/ml-training-image-classifier-using-tensorflow-object-detection-api/
print("[INFO] Model loaded.")

In [ ]:
colors_hash = {}
while True:
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()

    # Convert images to numpy arrays
    color_image = np.asanyarray(color_frame.get_data())
    scaled_size = (color_frame.width, color_frame.height)
    # expand image dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value
    image_expanded = np.expand_dims(color_image, axis=0)
    # Perform the actual detection by running the model with the image as input
    (boxes, scores, classes, num) = sess.run([detection_boxes, detection_scores, detection_classes, num_detections],
                                             feed_dict={image_tensor: image_expanded})

    boxes = np.squeeze(boxes)
    classes = np.squeeze(classes).astype(np.int32)
    scores = np.squeeze(scores)

    for idx in range(int(num)):
        class_ = classes[idx]
        score = scores[idx]
        box = boxes[idx]

        if class_ not in colors_hash:
            colors_hash[class_] = tuple(np.random.choice(range(256), size=3))

        if score > 0.6:
            left = int(box[1] * color_frame.width)
            top = int(box[0] * color_frame.height)
            right = int(box[3] * color_frame.width)
            bottom = int(box[2] * color_frame.height)

            p1 = (left, top)
            p2 = (right, bottom)
            # draw box
            r, g, b = colors_hash[class_]
            cv2.rectangle(color_image, p1, p2, (int(r), int(g), int(b)), 2, 1)

    cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
    cv2.imshow('RealSense', color_image)
    cv2.waitKey(1)

print("[INFO] stop streaming ...")
pipeline.stop()